In [ ]:
#Correlation Network Analysis

In [ ]:
#Gene network analysis is a method designed to identify sub-networks (modules) of correlated genes, which are likely to be co-expressed.
#This can be helpful in identification of sub-networks (modules) of genes that contribute to disease.
#In this example, we will cover how to create a pairwise correlation matrix of genes, as well as how to associate them with disease.

In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata
import networkx as nx
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
import json
import scipy
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from scipy.spatial.distance import pdist, squareform
from statsmodels.stats.multitest import multipletests

In [ ]:
#Load in the required data
datExpr = pd.read_csv('/data/mic_datExpr_pseudobulk.csv', index_col = 0)
metadata = pd.read_csv('/data/recode/mic_metadata_pseudobulk.csv', index_col = 0)

In [ ]:
datExpr

In [ ]:
metadata

In [ ]:
#Correlation is a statistical measure that describes the extent to which two variables change together. It indicates the strength and direction of a linear relationship between two variables.
#Correlation analysis for coexpression networks is a method used to study the relationships between genes or proteins by analysing their expression levels across various conditions, tissues, or time points.
#We will be exploring correlation based co-expression networks within this exercise.

In [ ]:
#Step 1: 
#You can use the corr function to calculate pairwise pearson correlations between the genes.
correlation_matrix = datExpr.corr()

In [ ]:
#Lets view the correlation matrix. You will see that there is a diagonal of the value 1, due to the calculating the correlation between the same gene.
correlation_matrix

In [ ]:
#Step 2:
#Now that we have the correlation matrix, we need to calculate the distance matrix.
#A distance matrix is a mathematical representation that captures the pairwise distances between a set of objects. 
#In hierarchical clustering, distance matrices are used to determine which objects to merge or split based on their pairwise distances.
#So, we will be using the distance matrix in order to calculate clusters between genes, which will form our networks. These are commonly also called communities.

# Perform hierarchical clustering using the dissimilarity values
distance_matrix = 1 - correlation_matrix  # Convert correlation to distance measure

#The linkage function is used to perform hierarchical clustering on the distance matrix. 
linkage_matrix = sch.linkage(distance_matrix, method='average')

# Plot the dendrogram
plt.figure(figsize=(10, 6))
dendrogram = sch.dendrogram(linkage_matrix)

#Plot the title, y axis and x axis lables also

plt.show()

In [ ]:
#Step 3:
#Due to all of the vast correlations, the networks will be very messy each having lots of edges with varying correlation strengths.
#Not all of these are of interest, such as weak correlations. So we would solely like to focus on the strong correlations.

# Define threshold for significant edges (you can adjust this based on your requirement)
threshold = 

# Threshold the correlation matrix to determine significant edges
significant_edges = correlation_matrix > threshold

In [ ]:
#Lets have a look at the significant_edges dataframe.
#You can now see that this produces a boolean dataframe which states whether the values are greater than the threshold or not.
significant_edges

In [ ]:
#Step 4:
#networkx is a python library designed for network analysis.
# Construct the network from significant edges.
G = nx.Graph()

# Loop through significant edges and add edges to the graph
for i in range(significant_edges.shape[0]):
    for j in range(significant_edges.shape[1]):
        if significant_edges.values[i, j]:
            gene1 = significant_edges.index[i]
            gene2 = significant_edges.columns[j]
            
            # Find the position of gene1 and gene2 in the correlation matrix
            i_corr = np.where(correlation_matrix.index == gene1)[0][0]
            j_corr = np.where(correlation_matrix.columns == gene2)[0][0]
            
            # Add the edge to the graph
            G.add_edge(gene1, gene2, weight=correlation_matrix.iloc[i_corr, j_corr])

# Display the number of nodes and edges in the graph
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

In [ ]:
#Step 5:
# Analyse the network
# For example, you can identify clusters/modules using community detection algorithms
communities = nx.algorithms.community.greedy_modularity_communities(G)

In [ ]:
communities

In [ ]:
# Initialise a new graph for community visualisation
community_graph = nx.Graph()

In [ ]:
# Add edges between communities based on their connectivity
for i, community in enumerate(communities):
    for node in community:
        community_graph.add_node(node)
        for neighbor in G.neighbors(node):
            if neighbor in community:
                community_graph.add_edge(node, neighbor, weight=G[node][neighbor]['weight'])


In [ ]:
# Initialise an empty list to store separated communities
separated_communities = []

# Iterate over the detected communities
for community in communities:
    # Convert the set of nodes into a list and append it to the list of lists
    separated_communities.append(list(community))

In [ ]:
# Print or use the separated communities as needed
print(separated_communities)

In [ ]:
#Check how many communities there are? How could you do this? (Hint: len function)

In [ ]:
#Step 6:
# Choose the index of the community you want to visualise
community_index =   # Change this to the index of the community you want to visualize

# Get the nodes in the selected community
selected_community = list(communities[community_index])

# Create a subgraph containing only the nodes and edges within the selected community
subgraph = G.subgraph(selected_community)

In [ ]:
# Check how many genes are within the chosen network. (Hint: check the number of nodes)
num_nodes = 
print(f"The number of nodes in the subgraph is: {num_nodes}")

In [ ]:
#Step 7:
# Visualise the subgraph
pos = nx.spring_layout(subgraph, k=0.8, iterations=20)  # You can use different layout algorithms
plt.figure(figsize=(14, 12))
nx.draw(subgraph, pos, with_labels=True, node_color='skyblue', node_size=100, edge_color='gray', linewidths=0.1, font_size=8)
plt.title('Community Visualisation')
plt.show()

In [ ]:
#Play around with different layouts and see how the visualisation is affected.
#For example, there is circular_layout. Check out https://networkx.org/documentation/stable/tutorial.html#drawing-graphs for more information.

In [ ]:
#Exercise Questions
#1. What other correlation measures could be used?
#2. What other distance measures could be used?
#3. Are there other community-based algorithms that could be used instead?
#4. Show a worked through example testing different correlation measures, distance measures and community algorithms
#and see how this affects the networks produced.

In [ ]:
#Answers:
#1.
#2.
#3.
#4.